In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import datetime,gc,math
import random
%matplotlib inline
import gc
import lightgbm as lgb
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder,MinMaxScaler, StandardScaler
from sklearn.metrics import recall_score, auc, accuracy_score, f1_score, precision_score, classification_report, roc_auc_score,mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.feature_selection import VarianceThreshold

In [2]:
def active_time_transform(df):
    df['role_created_active_time'].fillna('"0-8,0","8-12,0","12-14,0","14-18,0","18-24,0"', inplace=True)
    df['role_created_active_time'] = df['role_created_active_time'].apply(lambda x:str(x)[1:-1])
    temp = df['role_created_active_time'].str.split(',',expand=True).iloc[:,[1,3,5,7,9]].rename(columns={1:'active_0-8',
                                        3:'active_8-12',5:'active_12-14',7:'active_14-18',9:'active_18-24'})
    for i in range(len(temp.columns)):
        temp.iloc[:,i] = temp.iloc[:,i].apply(lambda x:str(x)[:-1])
    df = df.join(temp).drop(['role_created_active_time'], axis=1)
    return df

def pay_grade_transform(df):
    df['pay_grade'].fillna('[0,0,0,0,0,0,0]',inplace=True)
    df['pay_grade'] = df['pay_grade'].apply(lambda x:str(x)[1:-1])
    temp = df['pay_grade'].str.split(',',expand=True).rename(columns={0:'pay_grade_1',
                1:'pay_grade_2',2:'pay_grade_3',3:'pay_grade_4',4:'pay_grade_5',5:'pay_grade_6',6:'pay_grade_7'})
    df = df.join(temp).drop(['pay_grade'], axis=1)
    return df

In [3]:
%%time
role_info = pd.read_csv('./data/mr_role_4d.csv')
role_info.drop_duplicates(subset=['user_id','cp_server_no','cp_role_id'],inplace=True)
role_info = active_time_transform(role_info)
role_info = pay_grade_transform(role_info)

Wall time: 1min 59s


In [4]:
# 删除2020-7-19日前30天的数据
role_info['create_role_time'] = pd.to_datetime(role_info['create_role_time'], format='%Y-%m-%d %H:%M:%S')
role_info.drop(role_info[role_info['create_role_time'] > (role_info['create_role_time'].max() + 
                                                          datetime.timedelta(-30))].index,inplace=True)

In [5]:
# 删除没有心跳数据的样本(1、没有登录;2、没有拿到心跳)
role_info.dropna(subset=['role_created_login_num'], inplace=True)
role_info.dropna(subset=['role_created_online'], inplace=True)
role_info['pay_num'].fillna(0, inplace=True)
role_info['pay_sum'].fillna(0, inplace=True)
role_info['role_created_active'] = role_info['role_created_active'].clip(0,4)
role_info['pay_rate'] = role_info['pay_num'] / (role_info['role_created_active'] + 1e-4)
role_info['pay_avg'] = role_info['pay_sum'] / (role_info['pay_num'] + 1e-4)

In [6]:
role_created_30_pay_sum = pd.read_csv('./data/role_created_30_pay_sum.csv',index_col=0)
# 对连接的关键字进行字符化或者整型化的调整
role_created_30_pay_sum['user_id'] = role_created_30_pay_sum['user_id'].astype(str)
role_created_30_pay_sum['mgame_id'] = role_created_30_pay_sum['mgame_id'].astype(str)
role_created_30_pay_sum['cp_server_no'] = role_created_30_pay_sum['cp_server_no'].astype(str)
role_created_30_pay_sum['cp_role_id'] = role_created_30_pay_sum['cp_role_id'].astype(str)
role_info['user_id'] = role_info['user_id'].astype(str)
role_info['mgame_id'] = role_info['mgame_id'].astype(str)
role_info['cp_server_no'] = role_info['cp_server_no'].astype(str)
role_info['cp_role_id'] = role_info['cp_role_id'].astype(str)
role_created_30_pay_sum.drop_duplicates(subset=['user_id','cp_server_no','mgame_id','cp_role_id'],inplace=True)
role_info.drop_duplicates(subset=['user_id','cp_server_no','mgame_id','cp_role_id'],inplace=True)
role_info = pd.merge(role_info,role_created_30_pay_sum,on=['user_id','mgame_id','cp_server_no','cp_role_id'],how='left',validate='one_to_one')

In [7]:
select_features = ['user_id','cp_server_no','cp_role_id','role_created_login_num',
       'role_created_active', 'role_created_online',
       'max_role_level', 'ip_num',
       'pay_num', 'pay_sum', 'active_0-8', 'active_8-12', 'active_12-14',
       'active_14-18', 'active_18-24', 'pay_grade_1', 'pay_grade_2',
       'pay_grade_3', 'pay_grade_4', 'pay_grade_5', 'pay_grade_6',
       'pay_rate', 'pay_avg','role_created_30_pay_sum']
role_info = role_info[select_features]
role_info['role_created_30_pay_sum'].fillna(0,inplace=True)
col_list = ['active_0-8','active_8-12','active_12-14','active_14-18','active_18-24','pay_grade_1','pay_grade_2',
            'pay_grade_3','pay_grade_4','pay_grade_5','pay_grade_6']
for col in col_list:
    role_info[col] = pd.to_numeric(role_info[col], errors='coerce')

In [11]:
role_info.to_pickle('./role_info_4d.pkl')

In [8]:
df_train, df_test = train_test_split(role_info,test_size=0.3)
df_train_pay = df_train[df_train['pay_sum']>0]
df_train_nopay = df_train[df_train['pay_sum']==0]

In [9]:
target = df_train_pay['role_created_30_pay_sum']
features = df_train_pay.drop(['role_created_30_pay_sum','user_id','cp_server_no','cp_role_id'], axis=1)
target_ln = np.log1p(target)
X_val, x_test, Y_val, y_test = train_test_split(features,target_ln, test_size=0.3)

In [10]:
# lgb的调参与交叉验证
train_data = lgb.Dataset(X_val, label=Y_val)
val_data = lgb.Dataset(x_test, label=y_test, reference=train_data)

# 设置参数初始值，不含交叉验证参数
print('设置参数')
params = {
    'boosting_type':'gbdt',
    'objective':'regression',
    'metric':'rmse',
    'nthread':4,
    'learning_rate':0.05
}

# 交叉验证（调参）
print('交叉验证')
min_rmse = float('5')
best_params = {}

# 准确率
print('调参1：提高准确率')
for num_leaves in range(5, 100, 5):
    for max_depth in range(3, 8, 1):
        params['num_leaves'] = num_leaves
        params['max_depth'] = max_depth
        
        cv_results = lgb.cv(
                            params,
                            train_data,
                            nfold=3,
                            stratified=False,
                            early_stopping_rounds=10,
                            verbose_eval=50     
                            )
        mean_rmse = pd.Series(cv_results['rmse-mean']).min()
        boost_rounds = pd.Series(cv_results['rmse-mean']).idxmin()
        
        if mean_rmse <= min_rmse:
            min_rmse = mean_rmse
            best_params['num_leaves'] = num_leaves
            best_params['max_depth'] = max_depth

if 'num_leaves' and 'max_depth' in best_params.keys():
    params['num_leaves'] = best_params['num_leaves']
    params['max_depth'] = best_params['max_depth']

# 过拟合
print('调参2：降低过拟合')
min_rmse = float('5')
for max_bin in range(5, 256, 10):
    for min_data_in_leaf in range(1, 102, 10):
        params['max_bin'] = max_bin
        params['min_data_in_leaf'] = min_data_in_leaf
        
        cv_results = lgb.cv(
                            params,
                            train_data,
                            nfold=3,
                            stratified=False,
                            early_stopping_rounds=10,
                            verbose_eval=50 
                            )
        mean_rmse = pd.Series(cv_results['rmse-mean']).min()
        boost_rounds = pd.Series(cv_results['rmse-mean']).idxmin()
        
        if mean_rmse <= min_rmse:
            min_rmse = mean_rmse
            best_params['min_data_in_leaf'] = min_data_in_leaf
            best_params['max_bin'] = max_bin
            
if 'max_bin' and 'min_data_in_leaf' in best_params.keys():
    params['max_bin'] = best_params['max_bin']
    params['min_data_in_leaf'] = best_params['min_data_in_leaf']

print('调参3：降低过拟合')
min_rmse = float('5')
for feature_fraction in [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    for bagging_fraction in [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
        for bagging_freq in range(0,50,5):
            params['feature_fraction'] = feature_fraction
            params['bagging_fraction'] = bagging_fraction
            params['bagging_freq'] = bagging_freq

            cv_results = lgb.cv(
                            params,
                            train_data,
                            nfold=3,
                            stratified=False,
                            early_stopping_rounds=10,
                            verbose_eval=50
                            )
            mean_rmse = pd.Series(cv_results['rmse-mean']).min()
            boost_rounds = pd.Series(cv_results['rmse-mean']).idxmin()
        
            if mean_rmse <= min_rmse:
                min_rmse = mean_rmse
                best_params['feature_fraction'] = feature_fraction
                best_params['bagging_fraction'] = bagging_fraction
                best_params['bagging_freq'] = bagging_freq

if 'feature_fraction' and 'bagging_fraction' and 'bagging_freq' in best_params.keys():
    params['feature_fraction'] = best_params['feature_fraction']
    params['bagging_fraction'] = best_params['bagging_fraction']
    params['bagging_freq'] = best_params['bagging_freq']

print('调参4：降低过拟合')
min_rmse = float('5')
for lambda_l1 in [1e-5,1e-3,1e-1,0.0,0.1,0.3,0.5,0.7,0.9,1.0]:
    for lambda_l2 in [1e-5,1e-3,1e-1,0.0,0.1,0.4,0.6,0.7,0.9,1.0]:
        params['lambda_l1'] = lambda_l1
        params['lambda_l2'] = lambda_l2
        cv_results = lgb.cv(
                            params,
                            train_data,
                            nfold=3,
                            stratified=False,
                            early_stopping_rounds=10,
                            verbose_eval=50
                            )
        mean_rmse = pd.Series(cv_results['rmse-mean']).min()
        boost_rounds = pd.Series(cv_results['rmse-mean']).idxmin()
        
        if mean_rmse <= min_rmse:
            min_rmse = mean_rmse
            best_params['lambda_l1'] = lambda_l1
            best_params['lambda_l2'] = lambda_l2
if 'lambda_l1' and 'lambda_l2' in best_params.keys():
    params['lambda_l1'] = best_params['lambda_l1']
    params['lambda_l2'] = best_params['lambda_l2']

print("调参5：降低过拟合2")
min_rmse = float('5')
for min_split_gain in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
    params['min_split_gain'] = min_split_gain
    
    cv_results = lgb.cv(
                            params,
                            train_data,
                            nfold=3,
                            stratified=False,
                            early_stopping_rounds=10,
                            verbose_eval=50    
                            )
    mean_rmse = pd.Series(cv_results['rmse-mean']).min()
    boost_rounds = pd.Series(cv_results['rmse-mean']).idxmin()
        
    if mean_rmse >= min_rmse:
        min_rmse = mean_rmse
        
    best_params['min_split_gain'] = min_split_gain
if 'min_split_gain' in best_params.keys():
    params['min_split_gain'] = best_params['min_split_gain']
print(best_params)

设置参数
交叉验证
调参1：提高准确率
[50]	cv_agg's rmse: 0.901669 + 0.00235644
[100]	cv_agg's rmse: 0.877434 + 0.00305436
[50]	cv_agg's rmse: 0.901669 + 0.00235644
[100]	cv_agg's rmse: 0.877328 + 0.00308394
[50]	cv_agg's rmse: 0.901669 + 0.00235644
[100]	cv_agg's rmse: 0.877328 + 0.00308394
[50]	cv_agg's rmse: 0.901669 + 0.00235644
[100]	cv_agg's rmse: 0.877328 + 0.00308394
[50]	cv_agg's rmse: 0.901669 + 0.00235644
[100]	cv_agg's rmse: 0.877328 + 0.00308394
[50]	cv_agg's rmse: 0.892432 + 0.00260673
[100]	cv_agg's rmse: 0.876014 + 0.00302843
[50]	cv_agg's rmse: 0.890137 + 0.00254891
[100]	cv_agg's rmse: 0.87552 + 0.00311232
[50]	cv_agg's rmse: 0.89004 + 0.00256542
[100]	cv_agg's rmse: 0.875352 + 0.00305137
[50]	cv_agg's rmse: 0.89004 + 0.00256542
[100]	cv_agg's rmse: 0.875326 + 0.00308032
[50]	cv_agg's rmse: 0.89004 + 0.00256542
[100]	cv_agg's rmse: 0.875341 + 0.00303682
[50]	cv_agg's rmse: 0.892432 + 0.00260673
[100]	cv_agg's rmse: 0.876014 + 0.00302843
[50]	cv_agg's rmse: 0.887586 + 0.00265729
[100]	c

[100]	cv_agg's rmse: 0.874818 + 0.00307471
[50]	cv_agg's rmse: 0.885919 + 0.00275134
[100]	cv_agg's rmse: 0.874758 + 0.00329514
[50]	cv_agg's rmse: 0.885935 + 0.00271225
[100]	cv_agg's rmse: 0.874698 + 0.00309325
[50]	cv_agg's rmse: 0.885941 + 0.00268182
[100]	cv_agg's rmse: 0.87464 + 0.00312396
[50]	cv_agg's rmse: 0.885955 + 0.002695
[100]	cv_agg's rmse: 0.874743 + 0.00306255
[50]	cv_agg's rmse: 0.886004 + 0.0026722
[100]	cv_agg's rmse: 0.87473 + 0.00307477
[50]	cv_agg's rmse: 0.885995 + 0.00269581
[100]	cv_agg's rmse: 0.874763 + 0.00304478
[50]	cv_agg's rmse: 0.886086 + 0.00281739
[100]	cv_agg's rmse: 0.874732 + 0.0031693
[50]	cv_agg's rmse: 0.886065 + 0.0027162
[100]	cv_agg's rmse: 0.875186 + 0.00305123
[50]	cv_agg's rmse: 0.885912 + 0.00270515
[100]	cv_agg's rmse: 0.874846 + 0.00305438
[50]	cv_agg's rmse: 0.885818 + 0.00277679
[100]	cv_agg's rmse: 0.874795 + 0.00320765
[50]	cv_agg's rmse: 0.885861 + 0.00278692
[100]	cv_agg's rmse: 0.874818 + 0.00307471
[50]	cv_agg's rmse: 0.885919 

[50]	cv_agg's rmse: 0.885818 + 0.00277679
[100]	cv_agg's rmse: 0.874795 + 0.00320765
[50]	cv_agg's rmse: 0.885861 + 0.00278692
[100]	cv_agg's rmse: 0.874818 + 0.00307471
[50]	cv_agg's rmse: 0.885919 + 0.00275134
[100]	cv_agg's rmse: 0.874758 + 0.00329514
[50]	cv_agg's rmse: 0.885935 + 0.00271225
[100]	cv_agg's rmse: 0.874698 + 0.00309325
[50]	cv_agg's rmse: 0.885941 + 0.00268182
[100]	cv_agg's rmse: 0.87464 + 0.00312396
[50]	cv_agg's rmse: 0.885955 + 0.002695
[100]	cv_agg's rmse: 0.874743 + 0.00306255
[50]	cv_agg's rmse: 0.886004 + 0.0026722
[100]	cv_agg's rmse: 0.87473 + 0.00307477
[50]	cv_agg's rmse: 0.885995 + 0.00269581
[100]	cv_agg's rmse: 0.874763 + 0.00304478
[50]	cv_agg's rmse: 0.886086 + 0.00281739
[100]	cv_agg's rmse: 0.874732 + 0.0031693
[50]	cv_agg's rmse: 0.886065 + 0.0027162
[100]	cv_agg's rmse: 0.875186 + 0.00305123
[50]	cv_agg's rmse: 0.885912 + 0.00270515
[100]	cv_agg's rmse: 0.874846 + 0.00305438
[50]	cv_agg's rmse: 0.885818 + 0.00277679
[100]	cv_agg's rmse: 0.874795 

[100]	cv_agg's rmse: 0.875186 + 0.00305123
[50]	cv_agg's rmse: 0.885912 + 0.00270515
[100]	cv_agg's rmse: 0.874846 + 0.00305438
[50]	cv_agg's rmse: 0.885818 + 0.00277679
[100]	cv_agg's rmse: 0.874795 + 0.00320765
[50]	cv_agg's rmse: 0.885861 + 0.00278692
[100]	cv_agg's rmse: 0.874818 + 0.00307471
[50]	cv_agg's rmse: 0.885919 + 0.00275134
[100]	cv_agg's rmse: 0.874758 + 0.00329514
[50]	cv_agg's rmse: 0.885935 + 0.00271225
[100]	cv_agg's rmse: 0.874698 + 0.00309325
[50]	cv_agg's rmse: 0.885941 + 0.00268182
[100]	cv_agg's rmse: 0.87464 + 0.00312396
[50]	cv_agg's rmse: 0.885955 + 0.002695
[100]	cv_agg's rmse: 0.874743 + 0.00306255
[50]	cv_agg's rmse: 0.886004 + 0.0026722
[100]	cv_agg's rmse: 0.87473 + 0.00307477
[50]	cv_agg's rmse: 0.885995 + 0.00269581
[100]	cv_agg's rmse: 0.874763 + 0.00304478
[50]	cv_agg's rmse: 0.886086 + 0.00281739
[100]	cv_agg's rmse: 0.874732 + 0.0031693
[50]	cv_agg's rmse: 0.886065 + 0.0027162
[100]	cv_agg's rmse: 0.875186 + 0.00305123
[50]	cv_agg's rmse: 0.885912 

[50]	cv_agg's rmse: 0.887946 + 0.00298268
[100]	cv_agg's rmse: 0.875379 + 0.00336027
[50]	cv_agg's rmse: 0.887793 + 0.0029735
[100]	cv_agg's rmse: 0.875637 + 0.00352764
[50]	cv_agg's rmse: 0.887818 + 0.0030157
[100]	cv_agg's rmse: 0.875394 + 0.00366342
[50]	cv_agg's rmse: 0.888021 + 0.00310001
[100]	cv_agg's rmse: 0.875757 + 0.00355252
[50]	cv_agg's rmse: 0.888339 + 0.00309009
[100]	cv_agg's rmse: 0.876187 + 0.00338257
[50]	cv_agg's rmse: 0.888437 + 0.00326532
[100]	cv_agg's rmse: 0.875946 + 0.00346767
[50]	cv_agg's rmse: 0.888161 + 0.0026757
[100]	cv_agg's rmse: 0.876081 + 0.00343489
[50]	cv_agg's rmse: 0.888381 + 0.00290916
[100]	cv_agg's rmse: 0.876409 + 0.0032983
[50]	cv_agg's rmse: 0.888164 + 0.00292031
[100]	cv_agg's rmse: 0.876172 + 0.00344553
[50]	cv_agg's rmse: 0.888282 + 0.00281
[100]	cv_agg's rmse: 0.876124 + 0.00356093
[50]	cv_agg's rmse: 0.887946 + 0.00298268
[100]	cv_agg's rmse: 0.875379 + 0.00336027
[50]	cv_agg's rmse: 0.887781 + 0.00308613
[100]	cv_agg's rmse: 0.875584 

[50]	cv_agg's rmse: 0.887337 + 0.00301196
[100]	cv_agg's rmse: 0.874969 + 0.00345914
[50]	cv_agg's rmse: 0.887305 + 0.00324933
[100]	cv_agg's rmse: 0.875146 + 0.00343098
[50]	cv_agg's rmse: 0.887317 + 0.00315147
[100]	cv_agg's rmse: 0.875142 + 0.0032678
[50]	cv_agg's rmse: 0.88712 + 0.0028809
[100]	cv_agg's rmse: 0.874959 + 0.00323574
[50]	cv_agg's rmse: 0.88712 + 0.0028809
[100]	cv_agg's rmse: 0.874959 + 0.00323574
[50]	cv_agg's rmse: 0.88712 + 0.0028809
[100]	cv_agg's rmse: 0.874959 + 0.00323574
[50]	cv_agg's rmse: 0.88712 + 0.0028809
[100]	cv_agg's rmse: 0.874959 + 0.00323574
[50]	cv_agg's rmse: 0.88712 + 0.0028809
[100]	cv_agg's rmse: 0.874959 + 0.00323574
[50]	cv_agg's rmse: 0.88712 + 0.0028809
[100]	cv_agg's rmse: 0.874959 + 0.00323574
[50]	cv_agg's rmse: 0.88712 + 0.0028809
[100]	cv_agg's rmse: 0.874959 + 0.00323574
[50]	cv_agg's rmse: 0.88712 + 0.0028809
[100]	cv_agg's rmse: 0.874959 + 0.00323574
[50]	cv_agg's rmse: 0.88712 + 0.0028809
[100]	cv_agg's rmse: 0.874959 + 0.00323574

[50]	cv_agg's rmse: 0.886503 + 0.00315774
[100]	cv_agg's rmse: 0.87505 + 0.00314871
[50]	cv_agg's rmse: 0.886824 + 0.00296948
[100]	cv_agg's rmse: 0.875509 + 0.0033043
[50]	cv_agg's rmse: 0.886398 + 0.00259949
[100]	cv_agg's rmse: 0.875527 + 0.00310779
[50]	cv_agg's rmse: 0.886941 + 0.00294394
[100]	cv_agg's rmse: 0.875798 + 0.00345454
[50]	cv_agg's rmse: 0.886799 + 0.00283827
[100]	cv_agg's rmse: 0.875549 + 0.00308067
[50]	cv_agg's rmse: 0.886724 + 0.00291928
[100]	cv_agg's rmse: 0.875909 + 0.00335837
[50]	cv_agg's rmse: 0.886351 + 0.00282538
[100]	cv_agg's rmse: 0.874745 + 0.00313506
[50]	cv_agg's rmse: 0.886262 + 0.00304185
[100]	cv_agg's rmse: 0.87461 + 0.00320416
[50]	cv_agg's rmse: 0.886372 + 0.00273359
[100]	cv_agg's rmse: 0.874936 + 0.00320826
[50]	cv_agg's rmse: 0.886549 + 0.00262342
[100]	cv_agg's rmse: 0.875097 + 0.00292447
[50]	cv_agg's rmse: 0.886546 + 0.00292397
[100]	cv_agg's rmse: 0.875144 + 0.00308723
[50]	cv_agg's rmse: 0.886795 + 0.00294402
[100]	cv_agg's rmse: 0.875

[50]	cv_agg's rmse: 0.886066 + 0.00302852
[100]	cv_agg's rmse: 0.875645 + 0.00348967
[50]	cv_agg's rmse: 0.886364 + 0.0026
[100]	cv_agg's rmse: 0.8757 + 0.00278128
[50]	cv_agg's rmse: 0.886123 + 0.00287799
[100]	cv_agg's rmse: 0.87562 + 0.00322139
[50]	cv_agg's rmse: 0.886924 + 0.00269374
[100]	cv_agg's rmse: 0.876374 + 0.00270675
[50]	cv_agg's rmse: 0.886759 + 0.00276483
[100]	cv_agg's rmse: 0.876148 + 0.00300917
[50]	cv_agg's rmse: 0.88662 + 0.00255675
[100]	cv_agg's rmse: 0.876276 + 0.00311042
[50]	cv_agg's rmse: 0.886776 + 0.00251855
[100]	cv_agg's rmse: 0.876691 + 0.00279382
[50]	cv_agg's rmse: 0.886718 + 0.00249956
[100]	cv_agg's rmse: 0.876881 + 0.00302079
[50]	cv_agg's rmse: 0.886853 + 0.00249431
[100]	cv_agg's rmse: 0.876663 + 0.00307337
[50]	cv_agg's rmse: 0.885941 + 0.00268182
[100]	cv_agg's rmse: 0.87464 + 0.00312396
[50]	cv_agg's rmse: 0.88569 + 0.00275665
[100]	cv_agg's rmse: 0.874729 + 0.00318774
[50]	cv_agg's rmse: 0.886019 + 0.00260448
[100]	cv_agg's rmse: 0.875056 + 0

[50]	cv_agg's rmse: 0.886519 + 0.00295817
[100]	cv_agg's rmse: 0.874724 + 0.00330318
[50]	cv_agg's rmse: 0.886603 + 0.00301519
[100]	cv_agg's rmse: 0.874739 + 0.00327727
[50]	cv_agg's rmse: 0.886388 + 0.0029949
[100]	cv_agg's rmse: 0.874631 + 0.00334071
[50]	cv_agg's rmse: 0.886388 + 0.00299491
[100]	cv_agg's rmse: 0.874631 + 0.00334071
[50]	cv_agg's rmse: 0.886535 + 0.00286292
[100]	cv_agg's rmse: 0.874765 + 0.00317133
[50]	cv_agg's rmse: 0.886388 + 0.0029949
[100]	cv_agg's rmse: 0.874631 + 0.00334071
[50]	cv_agg's rmse: 0.886535 + 0.00286292
[100]	cv_agg's rmse: 0.874765 + 0.00317133
[50]	cv_agg's rmse: 0.886507 + 0.00299133
[100]	cv_agg's rmse: 0.874638 + 0.00323023
[50]	cv_agg's rmse: 0.886637 + 0.00289016
[100]	cv_agg's rmse: 0.874887 + 0.00327957
[50]	cv_agg's rmse: 0.886549 + 0.00300317
[100]	cv_agg's rmse: 0.874839 + 0.00323742
[50]	cv_agg's rmse: 0.886579 + 0.00299745
[100]	cv_agg's rmse: 0.874874 + 0.00322637
[50]	cv_agg's rmse: 0.886549 + 0.00298134
[100]	cv_agg's rmse: 0.87

In [12]:
params = {

'boosting_type':'gbdt',
'objective':'regression',
'metric':'rmse',
'nthread':4,
'learning_rate':0.05,
"max_depth":7,
"num_leaves":20,
"max_bin":255,
"min_data_in_leaf":61,
"min_split_gain":1.0,
"feature_fraction": 0.7,
"bagging_fraction":0.6,
"bagging_freq":5,
"lambda_l1":0.0,
"lambda_l2":0.0,
"metric": "rmse",
}
train_data = lgb.Dataset(X_val, label=Y_val)
val_data = lgb.Dataset(x_test, label=y_test, reference=train_data)
lgb_r_cv = lgb.train(params, train_data, num_boost_round=8000, early_stopping_rounds=100, valid_sets=[train_data, val_data])

[1]	training's rmse: 1.7417	valid_1's rmse: 1.73246
Training until validation scores don't improve for 100 rounds
[2]	training's rmse: 1.67794	valid_1's rmse: 1.66929
[3]	training's rmse: 1.61808	valid_1's rmse: 1.60998
[4]	training's rmse: 1.56297	valid_1's rmse: 1.55524
[5]	training's rmse: 1.51055	valid_1's rmse: 1.5033
[6]	training's rmse: 1.46155	valid_1's rmse: 1.45483
[7]	training's rmse: 1.4158	valid_1's rmse: 1.40953
[8]	training's rmse: 1.37313	valid_1's rmse: 1.36736
[9]	training's rmse: 1.33347	valid_1's rmse: 1.32813
[10]	training's rmse: 1.29664	valid_1's rmse: 1.29169
[11]	training's rmse: 1.26232	valid_1's rmse: 1.25775
[12]	training's rmse: 1.23053	valid_1's rmse: 1.22629
[13]	training's rmse: 1.20111	valid_1's rmse: 1.1972
[14]	training's rmse: 1.1744	valid_1's rmse: 1.17079
[15]	training's rmse: 1.14967	valid_1's rmse: 1.14645
[16]	training's rmse: 1.12682	valid_1's rmse: 1.12392
[17]	training's rmse: 1.10602	valid_1's rmse: 1.10349
[18]	training's rmse: 1.08653	vali

[188]	training's rmse: 0.857206	valid_1's rmse: 0.872541
[189]	training's rmse: 0.85709	valid_1's rmse: 0.872543
[190]	training's rmse: 0.857056	valid_1's rmse: 0.872541
[191]	training's rmse: 0.856973	valid_1's rmse: 0.872494
[192]	training's rmse: 0.856903	valid_1's rmse: 0.872489
[193]	training's rmse: 0.856852	valid_1's rmse: 0.872476
[194]	training's rmse: 0.856807	valid_1's rmse: 0.87247
[195]	training's rmse: 0.856745	valid_1's rmse: 0.87247
[196]	training's rmse: 0.856662	valid_1's rmse: 0.872438
[197]	training's rmse: 0.856584	valid_1's rmse: 0.872396
[198]	training's rmse: 0.856495	valid_1's rmse: 0.872424
[199]	training's rmse: 0.856436	valid_1's rmse: 0.872429
[200]	training's rmse: 0.856355	valid_1's rmse: 0.872406
[201]	training's rmse: 0.856266	valid_1's rmse: 0.872445
[202]	training's rmse: 0.856172	valid_1's rmse: 0.872465
[203]	training's rmse: 0.856091	valid_1's rmse: 0.872469
[204]	training's rmse: 0.856024	valid_1's rmse: 0.872449
[205]	training's rmse: 0.855909	va

In [13]:
df_test_pay = df_test[df_test['pay_sum']>0]
df_test_nopay = df_test[df_test['pay_sum']==0]

In [14]:
df_test_part1 = df_test_nopay[['user_id','cp_server_no','cp_role_id','pay_num']].rename(columns=
                                                                                {'pay_num':'predict_30_pay'})

In [15]:
target_test = df_test_pay['role_created_30_pay_sum']
target_test_ln = np.log1p(target_test)
features_test = df_test_pay.drop(['role_created_30_pay_sum','user_id','cp_server_no','cp_role_id'], axis=1)
y_predict = lgb_r_cv.predict(features_test)
y_predict[y_predict<0] = 0
mse = mean_squared_error(np.expm1(y_predict), np.expm1(target_test_ln))
mae = mean_absolute_error(np.expm1(y_predict), np.expm1(target_test_ln))
rmse = mse ** 0.5
print(rmse)
print(mse)
print(mae)

2662.987166293997
7091500.64784653
502.65746420672707


In [17]:
sum(np.expm1(target_test_ln))/sum(np.expm1(y_predict))

1.3957359427040092

In [18]:
df_test_part2 = df_test_pay[['user_id','cp_server_no','cp_role_id']]
df_test_part2['predict_30_pay'] = np.expm1(y_predict) *1.39
pred = df_test_part1.append(df_test_part2)
predict_data = pd.merge(df_test[['user_id','cp_server_no','cp_role_id', 'role_created_30_pay_sum']],pred,on=['user_id','cp_server_no','cp_role_id'],how='left',validate='one_to_one')

In [19]:
mse = mean_squared_error(predict_data['predict_30_pay'], predict_data['role_created_30_pay_sum'])
mae = mean_absolute_error(predict_data['predict_30_pay'], predict_data['role_created_30_pay_sum'])
rmse = mse ** 0.5
print('测试集上的均方根误差:%.2f元'% rmse)
print('测试集上的平均绝对误差:%.2f元'% mae)
print('测试集前30天实际总的付费金额:%.2f元' % predict_data['role_created_30_pay_sum'].sum())
print('测试集前30天预测总的付费金额:%.2f元'% predict_data['predict_30_pay'].sum())
print('预测总金额准确率:',predict_data['predict_30_pay'].sum()/predict_data['role_created_30_pay_sum'].sum())

测试集上的均方根误差:401.47元
测试集上的平均绝对误差:13.38元
测试集前30天实际总的付费金额:29606866.00元
测试集前30天预测总的付费金额:28190332.41元
预测总金额准确率: 0.9521552334849998


In [20]:
import joblib
joblib.dump(lgb_r_cv,'./lgb_r_4d.pkl')

['./lgb_r_4d.pkl']